In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
import ConeccionesSucursales as cs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/'
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
mes = 20210101
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
fin_mes = pd.to_datetime('31/01/2021',dayfirst=True)
quetzal = 2.54
dolar = 19.8

In [3]:
#Lectura pagos Bradesco
book = os.listdir(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/')
pagos_brad = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/'+i+'')
    pagos_new = pagos_new.iloc[:,0:9]
    pagos_brad = pd.concat([pagos_brad,pagos_new],ignore_index=True)
    pagos_brad = pagos_brad.drop_duplicates()
pagos_brad = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad = pd.DataFrame(pagos_brad.to_records())
pagos_brad = pagos_brad.loc[pagos_brad['FECHA_PAGO']>=mes3]

In [4]:
#Lectura pagos Liverpool
book =  os.listdir(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/')
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
pagos_liv_new = pd.DataFrame()
for i in book:
    for j in sheet_liv:
        pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/'+i+'',sheet_name = str(j),converters={'Cuenta':str})
        pagos_new.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
        pagos_liv_new = pd.concat([pagos_liv_new,pagos_new],ignore_index=True)
        pagos_liv_new = pagos_liv_new.iloc[:,0:6]
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
    pagos_liv = pagos_liv.drop_duplicates()
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
pagos_liv = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum) 
pagos_liv = pd.DataFrame(pagos_liv.to_records())
pagos_liv = pagos_liv.loc[pagos_liv['Fecha Pago']>=mes3]

In [5]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [6]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

In [7]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

# Llamadas detallado Liverpool Bradesco

In [8]:
#Hacemos la consulta referente a las llamadas de Liverpool
llamadas_liv = cs.query_naucalpan('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^LIV)')
llamadas_liv.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id','list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
llamadas_liv.head(1)

,server_ip,uniqueid,call_type,call_date,campaign_id,campaign_name,list_id,list_name,lead_id,vendor_lead_id,...,call_time_short,call_time_hour,call_range_hour,dialstatus,hangup_cause,sip_hangup_cause,carrier,areacode,state,region
0,192.168.15.21,1609483969.1374923,Inbound,2021-01-01 00:52:50,1102_IN,LIVERPOOL_BLASTER,999,Default inbound list,34186185,3331298280,...,0 days 00:52:50,0,00:00 - 01:00,ANSWER,16,200 OK,INBOUND,333,JAL,OCCIDENTE


In [9]:
#Hacemos la consulta referente a las llamadas de Bradesco
llamadas_brad = cs.query_naucalpan('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^BRAD)')
llamadas_brad.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id','list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
llamadas_brad.head(1)

,server_ip,uniqueid,call_type,call_date,campaign_id,campaign_name,list_id,list_name,lead_id,vendor_lead_id,...,call_time_short,call_time_hour,call_range_hour,dialstatus,hangup_cause,sip_hangup_cause,carrier,areacode,state,region
0,192.168.15.21,1609502079.1374953,Inbound,2021-01-01 05:54:40,1001_IN,BRADESCO_CALL,999,Default inbound list,33306642,3326563987,...,0 days 05:54:40,5,05:00 - 06:00,ANSWER,16,200 OK,INBOUND,332,JAL,OCCIDENTE


In [10]:
llamadas_liv_brad = pd.concat([llamadas_liv,llamadas_brad],ignore_index=True)

In [11]:
llamadas_liv_brad['turno'] = np.where(llamadas_liv_brad['call_time_hour']<=14,'M','V')

In [12]:
#Obtenemos el performance bruto de cada gestor
control_gestores_bruto_brad = pd.pivot_table(llamadas_liv_brad,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_bruto_brad = to_dataframe(control_gestores_bruto_brad)
control_gestores_bruto_brad = control_gestores_bruto_brad.iloc[:,[0,1,4,2,3]]
control_gestores_bruto_brad.columns = ['Gestor','Turno','NumeroGestionesB','TiempoTotalSegB','PromedioLlamadaSegB']
control_gestores_bruto_brad['TiempoTotalMinB'] = [x/60 for x in control_gestores_bruto_brad['TiempoTotalSegB']]
control_gestores_bruto_brad

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB
0,,M,1,0,0.000000,0.000000
1,AGUILAR MACEDO VALERIA,M,1544,57752,37.404145,962.533333
2,AGUILAR MACEDO VALERIA,V,407,13011,31.968059,216.850000
3,AGUILAR RIOS DANIEL ALEJANDRO,M,4762,97642,20.504410,1627.366667
4,AGUILAR RIOS DANIEL ALEJANDRO,V,155,6013,38.793548,100.216667
...,...,...,...,...,...,...
113,VAZQUEZ SANCHEZ LINNETH BERENICE,V,731,44220,60.492476,737.000000
114,VEGA HERNANDEZ NATALIE,M,2742,189555,69.130197,3159.250000
115,VEGA HERNANDEZ NATALIE,V,556,41841,75.253597,697.350000
116,ZARZA NORIA JEFREY ALBERTO,M,1748,97512,55.784897,1625.200000


In [13]:
#Obtenemos el performance de las llamadas contestadas
llamadas_liv_brad2 = llamadas_liv_brad.loc[llamadas_liv_brad['status_name'].isin(['Contacto con titular','Promesa de pago','Mensaje con terceros','Mensaje con familiar','Negativa de pago','Contacto familiar','Agendar llamada'])]
control_gestores_llamadas_brad = pd.pivot_table(llamadas_liv_brad2,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_llamadas_brad = to_dataframe(control_gestores_llamadas_brad)
control_gestores_llamadas_brad = control_gestores_llamadas_brad.iloc[:,[0,1,4,2,3]]
control_gestores_llamadas_brad.columns = ['Gestor','Turno','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_llamadas_brad['TiempoTotalMin'] = [x/60 for x in control_gestores_llamadas_brad['TiempoTotalSeg']]
#control_gestores_llamadas_brad

In [14]:
control_liv_consol_turno = pd.merge(control_gestores_bruto_brad,control_gestores_llamadas_brad,how='left',on=['Gestor','Turno'])
control_liv_consol_turno.fillna(0,inplace=True)
control_liv_consol_turno.head(2)

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,,M,1,0,0.000000,0.000000,0.0,0.0,0.0,0.00
1,AGUILAR MACEDO VALERIA,M,1544,57752,37.404145,962.533333,75.0,7425.0,99.0,123.75


In [15]:
#Obtenemos el performance de las llamadas contestadas
control_gestores_total = pd.pivot_table(llamadas_liv_brad2,index=['full_name'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_total = to_dataframe(control_gestores_total)
control_gestores_total = control_gestores_total.iloc[:,[0,3,1,2]]
control_gestores_total.columns = ['Gestor','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_total['TiempoTotalMin'] = [x/60 for x in control_gestores_total['TiempoTotalSeg']]
control_gestores_total

,Gestor,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,AGUILAR MACEDO VALERIA,97,10062,103.731959,167.700000
1,AGUILAR RIOS DANIEL ALEJANDRO,10,312,31.200000,5.200000
2,ALANIZ DIAZ ALISON JOCELIN,38,6898,181.526316,114.966667
3,ALEJANDRA YESENIA HERNANDEZ BITE,69,11035,159.927536,183.916667
4,AMADOR FLORES LUIS ANGEL,82,13323,162.475610,222.050000
5,BALBUENA MACEDO EDGAR URIEL,64,14619,228.421875,243.650000
6,BARRETO OSEGUERA MARIA FERNANDA,148,18712,126.432432,311.866667
7,BERENICE GUZMAN JAIME,59,7868,133.355932,131.133333
8,BERNAL TREJO MARIA FERNANDA,164,22673,138.250000,377.883333
9,CARRANCO ZAMORA JAIME WALBERTO,114,22264,195.298246,371.066667


# Performance Liverpool Bradesco

In [17]:
#Hacemos la consulta referente al performance del Gestor
performance_all = cs.query_naucalpan('procesos_externos.all_rpt_agent_performance',mes,today)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [18]:
performance_liv_brad = performance_all.loc[(performance_all['user_group']=='LIVERPOOL') | (performance_all['user_group']=='BRADESCARD')]
performance_liv_brad[['ocupacion','adherencia']] = percent_tonumeric(performance_liv_brad,['ocupacion','adherencia'])

In [19]:
def to_hour(df,cols):
    df[cols] = [datetime.strptime(x,'%H:%M:%S') for x in df[cols]]
    return df
    
#performance_liv_brad['login_vici2'] = to_hour(performance_liv_brad,'login_vici')
#performance_liv_brad['logout_vici2'] = to_hour(performance_liv_brad,'logout_vici')

In [20]:
performance_liv_brad['hora_login'] = [int(x[:2]) for x in performance_liv_brad['login_vici']]
performance_liv_brad['hora_logout'] = [int(x[:2]) for x in performance_liv_brad['logout_vici']]
performance_liv_brad['turno'] = np.where((performance_liv_brad['hora_logout']<=14)&(performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']<=8),'M',
                                    np.where((performance_liv_brad['hora_logout']>17)&(performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']<=8),'V','Mixto'))

In [21]:
performance_liv_brad['horas_trabajadas'] = performance_liv_brad['hora_logout']-performance_liv_brad['hora_login']
performance_liv_brad['llamadas_hora'] = performance_liv_brad['llamadas']/performance_liv_brad['horas_trabajadas']
performance_liv_brad['llamadas_hora'] = performance_liv_brad['llamadas_hora'].round(2)
performance_liv_brad.head(1)

,fecha,usuario,nombre_usuario,user_group,llamadas,aht,ocupacion,adherencia,login_vici,logout_vici,...,capacitacion,sistemas,lagged,tipificando,*,hora_login,hora_logout,turno,horas_trabajadas,llamadas_hora
49,2021-01-02,AGUILARMAVA,AGUILAR MACEDO VALERIA,BRADESCARD,11,00:01:18,18.2,25.08,12:04:44,13:15:49,...,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,12,13,M,1,11.0


In [22]:
control_performance_brad_turno = pd.pivot_table(performance_liv_brad,index=['nombre_usuario','turno'],values=['llamadas','ocupacion','llamadas_hora'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_brad_turno = to_dataframe(control_performance_brad_turno)
control_performance_brad_turno = control_performance_brad_turno.iloc[:,[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16]]
control_performance_brad_turno.columns = ['Gestor','Turno','SumLlamadasDia','SumLlamadasHora','SumOcupacion','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion','DiasAsitencia','MinLlamadasDia','MinLlamadasHora','MinOcupacion','MaxLlamadasDia','MaxLlamadasHora','MaxOcupacion']
control_performance_brad_turno.head(1)

,Gestor,Turno,SumLlamadasDia,SumLlamadasHora,SumOcupacion,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion,DiasAsitencia,MinLlamadasDia,MinLlamadasHora,MinOcupacion,MaxLlamadasDia,MaxLlamadasHora,MaxOcupacion
0,AGUILAR MACEDO VALERIA,M,708,122.02,83.15,88.5,15.2525,10.39375,8,11,10.29,5.92,148,21.14,18.2


In [23]:
control_performance_brad_turno2 = control_performance_brad_turno[['Gestor','Turno','PromedioLlamadasHora','PromedioOcupacion']]

In [24]:
control_performance_brad = pd.pivot_table(performance_liv_brad,index=['nombre_usuario'],values=['llamadas','ocupacion','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_brad = to_dataframe(control_performance_brad)
control_performance_brad = control_performance_brad.iloc[:,[0,1,2,6,7]]
control_performance_brad.columns = ['Gestor','SumHorasTrabajadas','SumLlamadasDia','PromedioOcupacion','DiasTrabajados']
control_performance_brad['LlamadasHora'] = (control_performance_brad['SumLlamadasDia']/control_performance_brad['SumHorasTrabajadas']).round()
control_performance_brad.head(3)

,Gestor,SumHorasTrabajadas,SumLlamadasDia,PromedioOcupacion,DiasTrabajados,LlamadasHora
0,AGUILAR MACEDO VALERIA,149,1946,9.788824,17,13.0
1,AGUILAR RIOS DANIEL ALEJANDRO,12,482,15.415000,2,40.0
2,ALANIZ DIAZ ALISON JOCELIN,34,1677,25.107500,4,49.0


In [25]:
conjunto_performance_llamadas = pd.merge(control_gestores_total,control_performance_brad,on='Gestor',how='outer')
conjunto_performance_llamadas['ContactoHora'] = conjunto_performance_llamadas['NumeroGestiones']/conjunto_performance_llamadas['SumHorasTrabajadas']
conjunto_performance_llamadas = conjunto_performance_llamadas.iloc[:,[0,7,10,9,]]
conjunto_performance_llamadas

,Gestor,PromedioOcupacion,ContactoHora,LlamadasHora
0,AGUILAR MACEDO VALERIA,9.788824,0.651007,13.0
1,AGUILAR RIOS DANIEL ALEJANDRO,15.415000,0.833333,40.0
2,ALANIZ DIAZ ALISON JOCELIN,25.107500,1.117647,49.0
3,ALEJANDRA YESENIA HERNANDEZ BITE,NaN,NaN,NaN
4,AMADOR FLORES LUIS ANGEL,34.190909,1.576923,23.0
5,BALBUENA MACEDO EDGAR URIEL,37.652500,0.415584,23.0
6,BARRETO OSEGUERA MARIA FERNANDA,14.990000,1.465347,19.0
7,BERENICE GUZMAN JAIME,26.904118,0.437037,47.0
8,BERNAL TREJO MARIA FERNANDA,18.086000,2.129870,29.0
9,CARRANCO ZAMORA JAIME WALBERTO,26.276667,3.454545,32.0


# Gestion detallado Liverpool Bradesco

# Liverpool

In [26]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [27]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']

In [28]:
#Hacemos el consolidado 
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes.fillna('COBRANZA',inplace=True)
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])
#consolidado_liv_mes.head(3)

In [29]:
#Obtenemos los pagos totales por division
pagos_totales_liv = pd.pivot_table(pagos_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
#pagos_totales_liv = pagos_totales_liv2.iloc[:,[0,2,1]]
pagos_totales_liv.columns = ['Division','SumaPagos']
pagos_totales_liv.fillna(0,inplace=True)

In [30]:
#Para cada segmento se le asigna lo que lleva de cada uno
DKN00REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN00REM', 'SumaPagos'].item()
DKN02REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN02REM', 'SumaPagos'].item()
DKN04REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN04REM', 'SumaPagos'].item()
SA001REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA001REM', 'SumaPagos'].item()
SA002REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA002REM', 'SumaPagos'].item()

In [31]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
cof, arancel_cof,arancel_cof2 = 300000, .5, .5
sub, arancel_sub, arancel_sub2 = 300000, .15, .2
subdic, arancel_subdic, arancel_subdic2 = 350000, .16, .2
n2, arancel_n2, arancel_n22 = 650000, .4, .5
pren, arancel_pren, arancel_pren2 = 300000, .28, .28

In [32]:
#Funcion que asignará el arancel correspondiente
def arancel_liv(row):
    if row['Id'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),arancel_pren,arancel_pren2)
    elif row['Id'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),arancel_n2,arancel_n22)
    elif row['Id'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),arancel_cof,arancel_cof2)
    elif row['Id'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),arancel_sub,arancel_sub2)
    elif row['Id'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),arancel_subdic,arancel_subdic2)
    else:
        return 0

In [33]:
productos = [('DKN00REM','Liverpool','Penacional'),('DKN02REM','Liverpool Nacional','Nacional2'),('DKN04REM','Liverpool Cof','Cof'),('SA001REM','Liverpool','Suburbia'),('SA002REM','Liverpool','Dictamen')]
productos = pd.DataFrame(productos)
productos.columns = ['division','id','segmento']

In [34]:
#Aplicamos la funcion para obtener los aranceles
pagos_liv2 = pd.merge(pagos_liv,productos,how='left',left_on='Id',right_on='division')
pagos_liv2['Arancel'] = pagos_liv2.apply(arancel_liv, axis=1)

#Agregamos aranceles al consolidado
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)

In [35]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv2,how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
try:consolidado_pagos_liv['fecha_promesa'] = pd.to_datetime(consolidado_pagos_liv['fecha_promesa'],dayfirst=False)
except: consolidado_pagos_liv['fecha_promesa'] = consolidado_pagos_liv['fecha_promesa']
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
try:
    consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]
except: consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['dia_pago']
#consolidado_pagos_liv.head(3)

In [36]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z <= x+pd.Timedelta(3, unit='D')) & ((z >= x)), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]
#consolidado_pagos_liv.head(3)

In [37]:
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [38]:
#Obtenemos datos de los gestores de Liverpool
prom_liv_gestor_mes = pd.pivot_table(consolidado_pagos_liv,index=['nombre_usuario'],values=['credito','monto_promesa','PagoReal','MontoArancel'],aggfunc=['count',np.sum],dropna=False)
prom_liv_gestor_mes = pd.DataFrame(prom_liv_gestor_mes.to_records())
prom_liv_gestor_mes = prom_liv_gestor_mes.iloc[:,[0,4,1,7,6,5]]
prom_liv_gestor_mes.columns = ['Gestor','NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel']
prom_liv_gestor_mes.fillna(0,inplace=True)
prom_liv_gestor_mes.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel
0,AGUILAR RIOS DANIEL ALEJANDRO,12,5,26501.0,2021.0,808.4


# Bradesco

In [39]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad_mes['NUMERO_TARJETA'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['NUMERO_TARJETA','usuario','fecha_promesa','monto_promesa'])
gestion_brad_mes = gestion_brad_mes.loc[(gestion_brad_mes['fecha_promesa'].notnull())&(gestion_brad_mes['fecha_promesa']!='')]
gestion_brad_mes = gestion_brad_mes.loc[(gestion_brad_mes['fecha_promesa'].str[:4].map(int)<=2100)&(gestion_brad_mes['fecha_promesa'].str[:4].map(int)>=2000)]
gestion_brad_mes['fecha_promesa'] = pd.to_datetime(gestion_brad_mes['fecha_promesa'],dayfirst=False)

In [40]:
#Asignacion Bradesco
asignacion_brad_c0 = pd.read_csv(''+ruta+'/Bradesco/Asignacion_c0/CSV/ASIGNACION C0 '+mes2+'.csv')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [41]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
#consolidado_brad_mes.fillna('COBRANZA',inplace=True)
consolidado_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(consolidado_brad_mes['NUMERO_TARJETA'])

In [42]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [43]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [44]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

In [45]:
pagos_brad2 = pagos_brad
pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]

In [46]:
#Hacemos la union consolidado y pagos
consoliado_pagos_brad = pd.merge(consolidado_brad_mes,pagos_brad2,how='left',on=['NUMERO_TARJETA'])
consoliado_pagos_brad['fecha_promesa'] = pd.to_datetime(consoliado_pagos_brad['fecha_promesa'],dayfirst=False)

In [47]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consoliado_pagos_brad['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x)), y, 0) for x,y,z in zip(consoliado_pagos_brad['fecha_promesa'],consoliado_pagos_brad['MONTO'],consoliado_pagos_brad['FECHA_PAGO'])]
consoliado_pagos_brad['PagoUnico'] = [x.item() for x in consoliado_pagos_brad['PagoUnico']]

In [48]:
consoliado_pagos_brad['PagoReal'] = [x*y for x,y in zip(consoliado_pagos_brad['PagoUnico'],consoliado_pagos_brad['Arancel'])]
consoliado_pagos_brad2 = consoliado_pagos_brad.drop_duplicates(subset=['NUMERO_TARJETA','MONTO','FECHA_PAGO','usuario'])

In [49]:
pagos_totales_brad2 = pd.pivot_table(consoliado_pagos_brad,index=['usuario'],values=['monto_promesa','MONTO','PagoReal'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
# pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,3,1,6,4,5]]
# pagos_totales_brad2.columns = ['Gestor','NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel']
# pagos_totales_brad2.fillna(0,inplace=True)
#pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2.head(1)

,usuario,"('count', 'MONTO')","('count', 'PagoReal')","('count', 'monto_promesa')","('sum', 'MONTO')","('sum', 'PagoReal')","('sum', 'monto_promesa')"
0,AGUILARMAVA,71,71,113,108993.0,1171.06,122013.0


In [50]:
gestor_brad_liv = pd.concat([pagos_totales_brad2,prom_liv_gestor_mes],ignore_index=True)
gestor_final_brad_liv = pd.pivot_table(gestor_brad_liv,index=['Gestor'],values=['NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel'],aggfunc=np.sum)
gestor_final_brad_liv = pd.DataFrame(gestor_final_brad_liv.to_records())
gestor_final_brad_liv = gestor_final_brad_liv.iloc[:,[0,2,1,5,4,3]]
gestor_final_brad_liv['EfVolumen'] = gestor_final_brad_liv['NumeroPagos']/gestor_final_brad_liv['NumeroPromesas']
gestor_final_brad_liv['EfPagos'] = gestor_final_brad_liv['SumaPagos']/gestor_final_brad_liv['SumaPromesas']
gestor_final_brad_liv.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel,EfVolumen,EfPagos
0,AGUILAR RIOS DANIEL ALEJANDRO,12.0,5.0,26501.0,2021.0,808.4,0.416667,0.076261


In [51]:
gestor_final_brad_liv2 = gestor_final_brad_liv[['Gestor','SumaPagos','EfPagos']]
gestor_final_brad_liv2.head(1)

,Gestor,SumaPagos,EfPagos
0,AGUILAR RIOS DANIEL ALEJANDRO,2021.0,0.076261


In [52]:
final = pd.merge(gestor_final_brad_liv2,conjunto_performance_llamadas,how='outer',on='Gestor')
#final['PromedioLlamadasHora'] = final['PromedioLlamadasHora'].round()
final['PromedioOcupacion'] = final['PromedioOcupacion']/100
cols = ['PromedioOcupacion','EfPagos','LlamadasHora','ContactoHora']
#final[cols] = final[cols].round(2)
final.fillna(0,inplace=True)
final

,Gestor,SumaPagos,EfPagos,PromedioOcupacion,ContactoHora,LlamadasHora
0,AGUILAR RIOS DANIEL ALEJANDRO,2021.00,0.076261,0.154150,0.833333,40.0
1,ALEJANDRA YESENIA HERNANDEZ BITE,8928.00,0.148589,0.000000,0.000000,0.0
2,BARRETO OSEGUERA MARIA FERNANDA,52131.00,0.344619,0.149900,1.465347,19.0
3,BERENICE GUZMAN JAIME,19223.00,0.143819,0.269041,0.437037,47.0
4,CASTILLEJO CASTILLEJO DULCE MICHELLE,0.00,0.000000,0.000000,0.000000,0.0
5,CASTRO REZA IRVING YAEL,0.00,0.000000,0.411575,0.197080,23.0
6,COLULA CALIXTO MIGUEL ANGEL,20010.00,0.226416,0.403200,0.921348,37.0
7,CRUZ SALAZAR XIMENA,58148.00,0.458394,0.280612,0.137931,44.0
8,DANIEL PINEDA ALVARADO,0.00,0.000000,0.295519,0.409836,21.0
9,DIAZ DIAZ LEONOR,28461.00,0.136053,0.072361,0.230769,18.0


In [53]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/BradLiv/Reporte Gestor BradLiv '+now+'.xlsx',engine='xlsxwriter')
final.to_excel(writer,'BSC',index=False,header=True)
gestor_final_brad_liv2.to_excel(writer,'MontoRecuperado',index=False,header=True)
control_liv_consol_turno.to_excel(writer,'LlamadasDetallado',index=False,header=True)
control_performance_brad_turno.to_excel(writer,'PerformanceDetallado',index=False,header=True)
gestor_final_brad_liv.to_excel(writer,'PagosDetallado',index=False,header=True)

writer.save()